In [110]:
import RAG_utils
db = RAG_utils.access_lc_chroma_db('train_1000_200_all-MiniLM-L6-v2')
question = 'What does the name Madigan mean in Irish?'

system_message = """You are a helpful assistant. Answer the user's question in one sentence based on the provided context. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Do NOT start your response with "According to the provided context." """
user_message_template = """Context: {context} Question: {question}"""

# Retrieve the answer
docs = db.similarity_search(question, 11)
for i, doc in enumerate(docs):
    print('='*20, i, '='*20)
    print(doc.page_content)


context = RAG_utils.format_contexts([doc.page_content for doc in docs])
user_message = user_message_template.format(context=context, question=question)

answer = RAG_utils.gen_text_ollama(sys_msg=system_message, user_msg=user_message,options={'seed':0, 'temperature':0.01})
print(question)
print(answer)

==================== 0 ====================
edit* my ancestors were from Clan Ross
Family lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.
==================== 1 ====================
Challenge 10: Write a Story From a Movie You Like Hey, Everybody! Sorry it’s been so long. I’ve been working on this How To Train Your Dragon story a lot. […]
Challenge 4: Have two of your book characters from different book worlds meet. Write a short story about it. Writefury’s Characters Gilligan: From my first book: “Odd Team Out”. He […]
Thought I’d add a little bit of Riley Trivia this time. Madigan means “small dog” in Irish. Magnus means “big” in latin. Chapter 6 Training Shyloh After her […]
CHAPTER 4 Not a Slave A few days later, all the Irish had been sold. Riley had been sold to a cranky old lady named Freyda. In Freyda’s house, Riley […]
CHAPTER 2 Captured Riley opened her eyes and stared upward. Two seagulls squawked overh

In [1]:
from utils.db.es import search_data, search_vector
index_name = 'rag-dataset-12000-train-vector'
k = 5
question = 'What does the name Madigan mean in Irish?'
es_contexts = [context['_source']['sentence'] for context in search_vector(question, index_name, k)]

Current Working Directory: /Users/ryankish/Documents/Northwestern/Curriculum/Spring/MSAI 490 Practicum in Intelligent Systems/advanced_rag_team_twain
Project Path: /Users/ryankish/Documents/Northwestern/Curriculum/Spring/MSAI 490 Practicum in Intelligent Systems/advanced_rag_team_twain
{'elasticsearch': {'mikky': {'region': 'us-east-2', 'host': 'search-mikky-es-tkpsgqtkfdoy5kmpt4uprginoy.us-east-2.es.amazonaws.com', 'port': 443}, 'practicum': {'region': 'us-east-2', 'host': 'search-kcn5738-msai-practicum-5renmdtao2ribrht3jkdqbl75e.us-east-2.es.amazonaws.com', 'port': 443}}}


In [11]:
es_contexts

['edit* my ancestors were from Clan Ross\nFamily lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.',
 'Challenge 10: Write a Story From a Movie You Like Hey, Everybody! Sorry it’s been so long. I’ve been working on this How To Train Your Dragon story a lot. […]\nChallenge 4: Have two of your book characters from different book worlds meet. Write a short story about it. Writefury’s Characters Gilligan: From my first book: “Odd Team Out”. He […]\nThought I’d add a little bit of Riley Trivia this time. Madigan means “small dog” in Irish. Magnus means “big” in latin. Chapter 6 Training Shyloh After her […]\nCHAPTER 4 Not a Slave A few days later, all the Irish had been sold. Riley had been sold to a cranky old lady named Freyda. In Freyda’s house, Riley […]\nCHAPTER 2 Captured Riley opened her eyes and stared upward. Two seagulls squawked overhead. The ground rocked underneath her. She sat up slowly, her head throbbing.

# BGE

In [102]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()


pairs = [[question, context] for context in es_contexts]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    print(inputs)
    
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

threshold = 0
filtered_indices = torch.where(scores > threshold)[0]
filtered_values = scores[filtered_indices]
sorted_indices = torch.argsort(filtered_values, descending=True)
indices = filtered_indices[sorted_indices]
indices = indices.tolist()
sorted_relevant_contexts = [es_contexts[i] for i in indices]
sorted_relevant_contexts

{'input_ids': tensor([[    0,  4865, 14602,  ...,     1,     1,     1],
        [    0,  4865, 14602,  ...,  4293,  1449,     2],
        [    0,  4865, 14602,  ...,     1,     1,     1],
        [    0,  4865, 14602,  ...,     5,     2,     1],
        [    0,  4865, 14602,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
tensor([-9.4515,  3.5403, -9.4710, -9.3380, -6.0830])


['Challenge 10: Write a Story From a Movie You Like Hey, Everybody! Sorry it’s been so long. I’ve been working on this How To Train Your Dragon story a lot. […]\nChallenge 4: Have two of your book characters from different book worlds meet. Write a short story about it. Writefury’s Characters Gilligan: From my first book: “Odd Team Out”. He […]\nThought I’d add a little bit of Riley Trivia this time. Madigan means “small dog” in Irish. Magnus means “big” in latin. Chapter 6 Training Shyloh After her […]\nCHAPTER 4 Not a Slave A few days later, all the Irish had been sold. Riley had been sold to a cranky old lady named Freyda. In Freyda’s house, Riley […]\nCHAPTER 2 Captured Riley opened her eyes and stared upward. Two seagulls squawked overhead. The ground rocked underneath her. She sat up slowly, her head throbbing. Maddy barked happily, Riley […]']

In [69]:
tokenizer.passage_prefix

AttributeError: 'XLMRobertaTokenizerFast' object has no attribute 'passage_prefix'

In [60]:
tokenizer.decode(inputs['input_ids'][0])

'<s> What does the name Madigan mean in Irish?</s></s> edit* my ancestors were from Clan Ross Family lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.</s>'

In [46]:
tokenizer(question, padding=False, truncation=False, return_tensors='pt', max_length=512)['input_ids'][0]

tensor([     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
        120552,     32,      2])

In [41]:
inputs['input_ids'][0]

tensor([     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
        120552,     32,      2,      2,  27211,   1639,    759,      6,  60502,
         22230,   3542,   1295, 165854,  53434,  59745,    459,    107,     83,
           450,    642,   3542,   4727, 217258,      7,     23, 176637,      4,
          1284,   1608,   5281,   1810,      5,  52455,  14449,  43032, 109133,
            47,   5753,    136,  34739,     70,   9351,    159,    987,   1459,
             5,      2])

In [75]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

# Tokenize the question once
question_tokens = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=512)

# Tokenize the contexts
context_tokens = [tokenizer(context, padding=True, truncation=True, return_tensors='pt', max_length=512) for context in es_contexts]

# Concatenate the question tokens with each context token
pairs_tokens = [torch.cat((question_tokens['input_ids'], context_token['input_ids']), dim=1) for context_token in context_tokens]

# Create attention masks for the pairs
pairs_attention_masks = [torch.cat((question_tokens['attention_mask'], context_token['attention_mask']), dim=1) for context_token in context_tokens]

with torch.no_grad():
    scores = []
    for pair_input_ids, pair_attention_mask in zip(pairs_tokens, pairs_attention_masks):
        print(pair_input_ids)
        
        inputs = {
            'input_ids': pair_input_ids,
            'attention_mask': pair_attention_mask
        }
        score = model(**inputs, return_dict=True).logits.view(-1, ).float()
        scores.append(score)
    print(scores)

tensor([[     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
         120552,     32,      2,      0,  27211,   1639,    759,      6,  60502,
          22230,   3542,   1295, 165854,  53434,  59745,    459,    107,     83,
            450,    642,   3542,   4727, 217258,      7,     23, 176637,      4,
           1284,   1608,   5281,   1810,      5,  52455,  14449,  43032, 109133,
             47,   5753,    136,  34739,     70,   9351,    159,    987,   1459,
              5,      2]])
tensor([[     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
         120552,     32,      2,      0,  73630,    209,     12,    601,  18781,
             10,  30575,  28090,     10,  10348,   2583,  18852,  28240,      4,
          87769,  37873,     38,  90502,    442,     26,      7,   2809,    221,
           4989,      5,     87,     26,    272,   2809,  20697,     98,    903,
          11249,    717, 137879,  14804,  60884,  13765,     10,   5915,      5,
 

In [74]:
context_tokens[0]['input_ids']

tensor([[     0,  27211,   1639,    759,      6,  60502,  22230,   3542,   1295,
         165854,  53434,  59745,    459,    107,     83,    450,    642,   3542,
           4727, 217258,      7,     23, 176637,      4,   1284,   1608,   5281,
           1810,      5,  52455,  14449,  43032, 109133,     47,   5753,    136,
          34739,     70,   9351,    159,    987,   1459,      5,      2]])

In [54]:
tokenizer.decode([2])

'</s>'

In [48]:
pairs_attention_masks

[tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [35]:
es_contexts[0]

'edit* my ancestors were from Clan Ross\nFamily lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.'

In [42]:
inputs['input_ids'][0]# == pair_input_ids[0]

tensor([     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
        120552,     32,      2,      2,  27211,   1639,    759,      6,  60502,
         22230,   3542,   1295, 165854,  53434,  59745,    459,    107,     83,
           450,    642,   3542,   4727, 217258,      7,     23, 176637,      4,
          1284,   1608,   5281,   1810,      5,  52455,  14449,  43032, 109133,
            47,   5753,    136,  34739,     70,   9351,    159,    987,   1459,
             5,      2])

In [78]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

# Tokenize the question once
question_tokens = tokenizer(question, add_special_tokens=False)

# Prepare the pairs
pairs = [[question, context] for context in es_contexts]

input_ids = []
attention_masks = []

# Manually concatenate tokens with the [SEP] token in between
for pair in pairs:
    context_tokens = tokenizer(pair[1], add_special_tokens=False)
    
    # Concatenate question, [SEP] token, and context tokens
    input_id = question_tokens['input_ids'] + [tokenizer.sep_token_id] + context_tokens['input_ids']
    attention_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)

# Pad sequences to the same length
max_length = min(512, max(len(ids) for ids in input_ids))
input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in input_ids]
attention_masks = [mask + [0] * (max_length - len(mask)) for mask in attention_masks]

# Convert to PyTorch tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

with torch.no_grad():
    inputs = {'input_ids': input_ids, 'attention_mask': attention_masks}
    scores = model(**inputs, return_dict=True).logits.view(-1).float()
    print(scores)
input_ids[0]
tokenizer.decode(input_ids[0])

tensor([-9.4460,  4.2120, -9.3817, -9.0143, -6.9692])


'What does the name Madigan mean in Irish?</s> edit* my ancestors were from Clan Ross Family lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [89]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

# Tokenize the question once, adding start and end tokens
question_tokens = tokenizer(question, add_special_tokens=False)
question_tokens['input_ids'] = [tokenizer.cls_token_id] + question_tokens['input_ids'] + [tokenizer.sep_token_id]

# Prepare the pairs
pairs = [[question, context] for context in es_contexts]

input_ids = []
attention_masks = []

# Manually concatenate tokens with the [SEP] token in between
for pair in pairs:
    context_tokens = tokenizer(pair[1], add_special_tokens=False)
    context_tokens['input_ids'] = context_tokens['input_ids'] + [tokenizer.sep_token_id] # Add end token to context
    
    # Concatenate question, [SEP] token, and context tokens
    input_id = question_tokens['input_ids'] + context_tokens['input_ids']
    attention_mask = [1] * len(input_id)
    
    input_ids.append(input_id)
    attention_masks.append(attention_mask)

# Pad sequences to the same length
max_length = min(512, max(len(ids) for ids in input_ids))
input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in input_ids]
attention_masks = [mask + [0] * (max_length - len(mask)) for mask in attention_masks]

# Convert to PyTorch tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

with torch.no_grad():
    inputs = {'input_ids': input_ids, 'attention_mask': attention_masks}
    scores = model(**inputs, return_dict=True).logits.view(-1).float()
    print(scores)


tensor([-9.4276,  4.1190, -9.4661, -9.3947, -6.7785])


In [93]:
len(input_ids[1])

255

In [91]:
tokenizer.decode(input_ids[0])

'<s> What does the name Madigan mean in Irish?</s> edit* my ancestors were from Clan Ross Family lore is that we were MacDonalds in Scotland, but chased out. Many family members moved to York and took the name Sturdy.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [94]:
input_ids[0]

tensor([     0,   4865,  14602,     70,   9351,    276,  63592,  29459,     23,
        120552,     32,      2,  27211,   1639,    759,      6,  60502,  22230,
          3542,   1295, 165854,  53434,  59745,    459,    107,     83,    450,
           642,   3542,   4727, 217258,      7,     23, 176637,      4,   1284,
          1608,   5281,   1810,      5,  52455,  14449,  43032, 109133,     47,
          5753,    136,  34739,     70,   9351,    159,    987,   1459,      5,
             2,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1, 

In [62]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load pre-trained model and tokenizer
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Example texts
texts = ["I love this!", "I hate this..."]

# Tokenize the input text
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Forward pass, get logits
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Apply softmax to get probabilities
probabilities = torch.softmax(logits, dim=-1)
predictions = torch.argmax(probabilities, dim=-1)

print(probabilities)
print(predictions)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tensor([[1.2351e-04, 9.9988e-01],
        [9.9972e-01, 2.7708e-04]])
tensor([1, 0])


In [63]:
inputs

{'input_ids': tensor([[ 101, 1045, 2293, 2023,  999,  102,    0,    0],
        [ 101, 1045, 5223, 2023, 1012, 1012, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [65]:
tokenizer.decode(inputs['input_ids'][1])

'[CLS] i hate this... [SEP]'